In [ ]:
pip install talos

     |████████████████████████████████| 26.6MB 119kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.2.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.2.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
# Pemanggilan Pustaka
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import talos 
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam, RMSprop
from keras.activations import softmax, relu, tanh
from talos.utils import lr_normalizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Pendefinisian Modul Pembersihan Data
list_baku = pd.read_csv("/content/drive/My Drive/Data/list_kata_baku.csv", encoding='ISO-8859-1')

#Cleaning data
def clean_text(tweet): 
    # Convert to lower case
    tweet = tweet.lower()
    
    # Clean www.* or https?://*
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    # Clean @username
    tweet = re.sub('@[^\s]+','',tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    # Clean punctuation
    tweet = re.sub(r"[^\w\s]",' ', tweet)
    # Clean number
    tweet = re.sub(r"[\d-]",'', tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #trim
    #tweet = tweet.strip('\'"')

    # Clean per Words
    words = tweet.split()
    tokens=[]
    for ww in words:
        #split repeated word
        for w in re.split(r'[-/\s]\s*', ww):
            #replace two or more with two occurrences
            pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
            w = pattern.sub(r"\1\1", w)
            #check if the word cosists of two or more alphabets
            val = re.search(r"^[a-zA-Z][a-zA-Z][a-zA-Z]*$", w)
            #add tokens
            if(w in list_baku):
                x=np.array(list_baku[list_baku["spell"]==w]["word"])
                w=x[0]
                tokens.append(w.lower())
            elif(w == 'rt' or val is None):
                continue
            else:
                tokens.append(w.lower())
    tweet = " ".join(tokens)
    return tweet

In [ ]:
sd1 = pd.read_csv("/content/drive/My Drive/Data/Capres_skripsi.csv", usecols=["Sentimen", "Teks"], encoding='ISO-8859-1')

sd1['Sentimen'] = sd1['Sentimen'].replace(-1,0)
sd1['Teks'] = sd1['Teks'].map(lambda x: clean_text(x))
sd1 = sd1[sd1['Teks'].apply(lambda x: len(x.split()) >=1)]

tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sd1['Teks'])

sequences = tokenizer.texts_to_sequences(sd1['Teks'])
sd1_data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=80)
sd1_labels = np.array(sd1['Sentimen'])
sd1_labels = pd.get_dummies(sd1_labels)

sd1_data_train, sd1_data_test, sd1_labels_train, sd1_labels_test = train_test_split(sd1_data, sd1_labels, test_size=0.2, random_state=42, stratify= sd1_labels)

In [ ]:
sd1_data_train.shape

(1508, 80)

In [ ]:
vocabulary_size = len(tokenizer.word_index) + 1 
sequence_length = sd1_data.shape[1]                   
batch_size      = 32 
epochs          = 150      

In [ ]:
talos_optimal_CNN=pd.DataFrame(columns=['epochs_optimal','val_loss_optimal','val_acc_optimal'])
def CNN_model(x_train, y_train, x_test, y_test, params):
    checkpoint_filepath = 'C:/Users/DSGroupUI/Desktop/SKRIPSI FARHA/SKRIPSI/Program/CNN'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                   save_weights_only=True,
                                                                   monitor='val_loss',
                                                                   mode='min',
                                                                   save_best_only=True)
    inputs = keras.layers.Input(shape=(sequence_length,))
    embedding = keras.layers.Embedding(vocabulary_size, params['embd_size'])(inputs)
    conv = keras.layers.Conv1D(params['filter'], params['region_size'], activation='relu',kernel_regularizer=keras.regularizers.l2(params['l2_cnn']))(embedding)
    pool = keras.layers.GlobalMaxPooling1D()(conv)
    output = keras.layers.Dense(2, activation='softmax',kernel_regularizer= keras.regularizers.l2(params['l2_dense']))(pool)
    model  = keras.models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    output = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_data=(x_test, y_test),
                    verbose=0,
                    callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    epoch = int(output.history['val_loss'].index(min(output.history['val_loss']))+1)
    loss, acc = model.evaluate(x_test, y_test)
    talos_optimal_CNN.loc[len(talos_optimal_CNN)] = [epoch,loss,acc]
    return output, model

In [ ]:
p1 = {'embd_size':[64,100,128],
      'filter':[200,250,300],
      'region_size':[3,4,5],
      'l2_cnn':[0.001,0.01],
      'l2_dense':[0.001,0.01]}

In [ ]:
scan_object = talos.Scan(x=sd1_data_train,
                         y=sd1_labels_train,
                         params=p1,
                         model=CNN_model,
                         experiment_name='CNN_model',
                         x_val= sd1_data_test,
                         y_val= sd1_labels_test,
                         fraction_limit=0.1,
                         random_method= 'quantum')

  0%|          | 0/10 [00:00<?, ?it/s]

12/12 [==============================] - 0s 7ms/step - loss: 0.3856 - accuracy: 0.8753


 10%|█         | 1/10 [02:58<26:42, 178.10s/it]

12/12 [==============================] - 0s 15ms/step - loss: 0.3680 - accuracy: 0.8621


 20%|██        | 2/10 [11:23<36:49, 276.22s/it]

...

 80%|████████  | 8/10 [36:18<08:10, 245.11s/it]

12/12 [==============================] - 0s 10ms/step - loss: 0.3895 - accuracy: 0.8674


 90%|█████████ | 9/10 [41:57<04:33, 273.31s/it]

12/12 [==============================] - 0s 12ms/step - loss: 0.3854 - accuracy: 0.8647


100%|██████████| 10/10 [48:42<00:00, 292.23s/it]


In [ ]:
hasil_CNN = pd.concat([scan_object.data,talos_optimal_CNN],axis=1)
hasil_CNN.sort_values('val_loss_optimal')

,start,end,duration,round_epochs,loss,accuracy,val_loss,val_accuracy,embd_size,filter,l2_cnn,l2_dense,region_size,epochs_optimal,val_loss_optimal,val_acc_optimal
7,11/26/20-105655,11/26/20-105802,66.992679,150,0.056004,0.996684,0.359595,0.864721,128,300,0.010,0.001,5,112.0,0.355254,0.864721
9,11/26/20-105903,11/26/20-110002,58.666491,150,0.057751,0.997347,0.361480,0.864721,100,200,0.001,0.010,5,134.0,0.359313,0.870027
4,11/26/20-105406,11/26/20-105509,62.503014,150,0.027449,0.997347,0.392894,0.864721,128,200,0.001,0.001,5,63.0,0.360390,0.877984
8,11/26/20-105803,11/26/20-105903,60.310956,150,0.028611,0.997347,0.383769,0.846154,100,300,0.001,0.001,4,55.0,0.371215,0.859416
6,11/26/20-105608,11/26/20-105655,47.676387,150,0.031819,0.998011,0.380491,0.862069,64,200,0.001,0.001,5,65.0,0.371949,0.864721
1,11/26/20-105107,11/26/20-105210,63.414753,150,0.060763,0.995358,0.388880,0.864721,128,300,0.010,0.001,3,67.0,0.382999,0.870027
2,11/26/20-105210,11/26/20-105304,54.074642,150,0.029176,0.998011,0.413842,0.854111,100,200,0.001,0.001,4,54.0,0.388231,0.862069
0,11/26/20-105006,11/26/20-105107,60.374964,150,0.029659,0.997347,0.397905,0.856764,100,300,0.001,0.001,3,60.0,0.391110,0.856764
3,11/26/20-105305,11/26/20-105406,61.444889,150,0.055656,0.997347,0.392692,0.859416,128,200,0.001,0.010,3,144.0,0.392491,0.862069
5,11/26/20-105509,11/26/20-105607,58.721493,150,0.064193,0.995358,0.404204,0.854111,100,300,0.010,0.001,3,65.0,0.400294,0.854111


In [ ]:
talos_optimal_LSTM=pd.DataFrame(columns=['epochs_optimal','val_loss_optimal','val_acc_optimal'])
def LSTM_model(x_train, y_train, x_test, y_test, params):
    checkpoint_filepath = 'C:/Users/DSGroupUI/Desktop/SKRIPSI FARHA/SKRIPSI/Program/LSTM'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    inputs = keras.layers.Input(shape=(sequence_length,))
    embedding = keras.layers.Embedding(vocabulary_size, params['embd_size'])(inputs)
    LSTM = keras.layers.LSTM(params['units'],kernel_regularizer= keras.regularizers.l2(params['l2_ker']), 
                              recurrent_regularizer= keras.regularizers.l2(params['l2_rec']))(embedding)
    output = keras.layers.Dense(2, activation='softmax',kernel_regularizer= keras.regularizers.l2(params['l2_dense']))(LSTM)
    model  = keras.models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    output = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_data=(x_test, y_test),
                    verbose=0,
                    callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    epoch = int(output.history['val_loss'].index(min(output.history['val_loss']))+1)
    loss, acc = model.evaluate(x_test, y_test)
    talos_optimal_LSTM.loc[len(talos_optimal_LSTM)] = [epoch,loss,acc]
    return output, model

In [ ]:
p2 = {'embd_size':[64,100,128],
      'units':[100,150,200],
      'l2_ker':[0.001,0.01],
      'l2_rec':[0.001,0.01],
      'l2_dense':[0.001,0.01]}

In [ ]:
scan_object2 = talos.Scan(x=sd1_data_train,
                         y=sd1_labels_train,
                         params=p2,
                         model=LSTM_model,
                         experiment_name='LSTM_model',
                         x_val= sd1_data_test,
                         y_val= sd1_labels_test,
                         fraction_limit=0.1,
                         random_method= 'quantum')

  0%|          | 0/7 [00:00<?, ?it/s]

12/12 [==============================] - 0s 4ms/step - loss: 0.4149 - accuracy: 0.8568


 14%|█▍        | 1/7 [01:10<07:02, 70.40s/it]

12/12 [==============================] - 0s 4ms/step - loss: 0.4631 - accuracy: 0.8408


 29%|██▊       | 2/7 [02:19<05:49, 69.91s/it]

...

 71%|███████▏  | 5/7 [06:20<02:35, 77.59s/it]

12/12 [==============================] - 0s 4ms/step - loss: 0.4615 - accuracy: 0.8594


 86%|████████▌ | 6/7 [07:40<01:18, 78.08s/it]

12/12 [==============================] - 0s 4ms/step - loss: 0.5026 - accuracy: 0.8541


100%|██████████| 7/7 [09:05<00:00, 77.95s/it]


In [ ]:
hasil_LSTM = pd.concat([scan_object2.data,talos_optimal_LSTM],axis=1)
hasil_LSTM.sort_values('val_loss_optimal')

,start,end,duration,round_epochs,loss,accuracy,val_loss,val_accuracy,embd_size,l2_dense,l2_ker,l2_rec,units,epochs_optimal,val_loss_optimal,val_acc_optimal
0,11/26/20-110003,11/26/20-110113,70.259511,150,0.045042,0.998011,0.662526,0.848806,64,0.001,0.001,0.001,200,20.0,0.414932,0.856764
4,11/26/20-110457,11/26/20-110623,85.957874,150,0.058730,0.998011,0.509944,0.862069,128,0.010,0.001,0.010,100,93.0,0.437084,0.867374
5,11/26/20-110623,11/26/20-110742,79.053521,150,0.072266,0.996684,0.588253,0.856764,100,0.010,0.001,0.001,200,80.0,0.461514,0.859416
1,11/26/20-110113,11/26/20-110222,68.603365,150,0.036996,0.998674,0.497484,0.880637,64,0.001,0.001,0.010,100,131.0,0.463072,0.840849
2,11/26/20-110222,11/26/20-110339,77.456944,150,0.075623,0.997347,0.568671,0.851459,100,0.010,0.001,0.001,200,122.0,0.469062,0.832891
3,11/26/20-110339,11/26/20-110457,77.654306,150,0.056089,0.997347,0.660038,0.843501,100,0.001,0.010,0.001,150,41.0,0.477933,0.840849
6,11/26/20-110743,11/26/20-110908,85.406888,150,0.083381,0.994695,0.623016,0.854111,128,0.010,0.010,0.001,100,102.0,0.502600,0.854111


In [ ]:
talos_optimal_CNNLSTM=pd.DataFrame(columns=['epochs_optimal','val_loss_optimal','val_acc_optimal'])
def CNNLSTM_model(x_train, y_train, x_test, y_test, params):
    checkpoint_filepath = 'C:/Users/DSGroupUI/Desktop/SKRIPSI FARHA/SKRIPSI/Program/CNNLSTM'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    inputs = keras.layers.Input(shape=(sequence_length,))
    embedding = keras.layers.Embedding(vocabulary_size, params['embd_size'])(inputs)
    conv = keras.layers.Conv1D(params['filter'], params['region_size'], activation='relu',kernel_regularizer= keras.regularizers.l2(params['l2_cnn']))(embedding)
    pool = keras.layers.MaxPooling1D(2)(conv)
    LSTM = keras.layers.LSTM(params['units'],kernel_regularizer= keras.regularizers.l2(params['l2_ker']), 
                              recurrent_regularizer= keras.regularizers.l2(params['l2_rec']))(pool)
    output = keras.layers.Dense(2, activation='softmax',kernel_regularizer= keras.regularizers.l2(params['l2_dense']))(LSTM)
    model  = keras.models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    output = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_data=(x_test, y_test),
                    verbose=0,
                    callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    epoch = int(output.history['val_loss'].index(min(output.history['val_loss']))+1)
    loss, acc = model.evaluate(x_test, y_test)
    talos_optimal_CNNLSTM.loc[len(talos_optimal_CNNLSTM)] = [epoch,loss,acc]
    return output, model

In [ ]:
p3 = {'embd_size':[64,100,128],
      'units':[100,150,200],
      'filter':[200,250,300],
      'region_size':[3,4,5],
      'l2_ker':[0.001,0.01],
      'l2_rec':[0.001,0.01],
      'l2_cnn':[0.001,0.01],
      'l2_dense':[0.001,0.01]}

In [ ]:
scan_object3 = talos.Scan(x=sd1_data_train,
                         y=sd1_labels_train,
                         params=p3,
                         model=CNNLSTM_model,
                         experiment_name='CNNLSTM_model',
                         x_val= sd1_data_test,
                         y_val= sd1_labels_test,
                         fraction_limit=0.1,
                         random_method= 'quantum')

  0%|          | 0/129 [00:00<?, ?it/s]

12/12 [==============================] - 0s 3ms/step - loss: 0.5029 - accuracy: 0.8488


  1%|          | 1/129 [01:12<2:34:13, 72.29s/it]

12/12 [==============================] - 0s 3ms/step - loss: 0.5361 - accuracy: 0.8700


  2%|▏         | 2/129 [02:35<2:39:47, 75.49s/it]

...

 98%|█████████▊| 127/129 [2:42:48<02:25, 72.92s/it]

12/12 [==============================] - 0s 4ms/step - loss: 0.4683 - accuracy: 0.8355


 99%|█████████▉| 128/129 [2:44:09<01:15, 75.50s/it]

12/12 [==============================] - 0s 3ms/step - loss: 0.5398 - accuracy: 0.8515


100%|██████████| 129/129 [2:45:22<00:00, 76.92s/it]


In [ ]:
hasil_CNNLSTM = pd.concat([scan_object3.data,talos_optimal_CNNLSTM],axis=1)
hasil_CNNLSTM.sort_values('val_loss_optimal')

,start,end,duration,round_epochs,loss,accuracy,val_loss,val_accuracy,embd_size,filter,l2_cnn,l2_dense,l2_ker,l2_rec,region_size,units,epochs_optimal,val_loss_optimal,val_acc_optimal
103,11/26/20-132056,11/26/20-132205,69.362630,150,0.046250,0.997347,0.453412,0.867374,64,200,0.010,0.001,0.001,0.010,3,200,94.0,0.414121,0.864721
26,11/26/20-114204,11/26/20-114314,70.018648,150,0.077745,0.995358,0.429662,0.870027,64,200,0.010,0.010,0.001,0.010,3,200,138.0,0.418606,0.864721
106,11/26/20-132439,11/26/20-132603,83.106136,150,0.043140,0.996684,0.439844,0.870027,100,250,0.010,0.001,0.001,0.010,3,100,107.0,0.423611,0.872679
118,11/26/20-134031,11/26/20-134156,85.027187,150,0.040635,0.998011,0.491466,0.875332,128,250,0.010,0.001,0.001,0.010,3,100,107.0,0.423819,0.864721
64,11/26/20-123016,11/26/20-123140,84.009645,150,0.064168,0.997347,0.452476,0.851459,128,200,0.010,0.010,0.001,0.010,5,150,103.0,0.426738,0.859416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,11/26/20-120725,11/26/20-120848,82.717543,150,0.068995,0.997347,0.629086,0.846154,128,300,0.001,0.010,0.010,0.001,4,100,71.0,0.557390,0.846154
89,11/26/20-130224,11/26/20-130345,81.170016,150,0.039092,0.997347,0.670196,0.854111,128,200,0.001,0.001,0.010,0.001,3,200,35.0,0.557675,0.862069
78,11/26/20-124841,11/26/20-124956,74.769809,150,0.067176,0.996684,0.659703,0.851459,100,300,0.010,0.001,0.010,0.001,5,100,46.0,0.567412,0.867374
79,11/26/20-124956,11/26/20-125102,65.798382,150,0.071584,0.996684,0.728782,0.843501,64,300,0.010,0.001,0.010,0.001,4,150,20.0,0.581089,0.875332


In [ ]:
hasil_CNNLSTM.sort_values('val_loss_optimal').head(10)

,start,end,duration,round_epochs,loss,accuracy,val_loss,val_accuracy,embd_size,filter,l2_cnn,l2_dense,l2_ker,l2_rec,region_size,units,epochs_optimal,val_loss_optimal,val_acc_optimal
103,11/26/20-132056,11/26/20-132205,69.362630,150,0.046250,0.997347,0.453412,0.867374,64,200,0.010,0.001,0.001,0.01,3,200,94.0,0.414121,0.864721
26,11/26/20-114204,11/26/20-114314,70.018648,150,0.077745,0.995358,0.429662,0.870027,64,200,0.010,0.010,0.001,0.01,3,200,138.0,0.418606,0.864721
106,11/26/20-132439,11/26/20-132603,83.106136,150,0.043140,0.996684,0.439844,0.870027,100,250,0.010,0.001,0.001,0.01,3,100,107.0,0.423611,0.872679
118,11/26/20-134031,11/26/20-134156,85.027187,150,0.040635,0.998011,0.491466,0.875332,128,250,0.010,0.001,0.001,0.01,3,100,107.0,0.423819,0.864721
64,11/26/20-123016,11/26/20-123140,84.009645,150,0.064168,0.997347,0.452476,0.851459,128,200,0.010,0.010,0.001,0.01,5,150,103.0,0.426738,0.859416
36,11/26/20-115455,11/26/20-115604,69.429133,150,0.047946,0.996684,0.442235,0.859416,64,200,0.010,0.001,0.001,0.01,3,150,75.0,0.427457,0.877984
125,11/26/20-134933,11/26/20-135051,77.972352,150,0.065863,0.996684,0.469214,0.862069,100,250,0.010,0.001,0.010,0.01,4,200,117.0,0.433045,0.859416
29,11/26/20-114551,11/26/20-114700,68.502466,150,0.077134,0.996684,0.457534,0.854111,64,250,0.010,0.010,0.001,0.01,4,200,110.0,0.433783,0.862069
105,11/26/20-132331,11/26/20-132439,67.755277,150,0.065821,0.998011,0.465154,0.870027,64,300,0.010,0.001,0.010,0.01,5,150,71.0,0.434338,0.870027
115,11/26/20-133659,11/26/20-133805,65.475282,150,0.051841,0.998011,0.453356,0.864721,64,250,0.001,0.010,0.001,0.01,3,150,131.0,0.435533,0.862069


In [25]:
talos_optimal_LSTMCNN=pd.DataFrame(columns=['epochs_optimal','val_loss_optimal','val_acc_optimal'])
def LSTMCNN_model(x_train, y_train, x_test, y_test, params):
    checkpoint_filepath = 'C:/Users/DSGroupUI/Desktop/SKRIPSI FARHA/SKRIPSI/Program/LSTMCNN'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    inputs = keras.layers.Input(shape=(sequence_length,))
    embedding = keras.layers.Embedding(vocabulary_size, params['embd_size'])(inputs)
    LSTM = keras.layers.LSTM(params['units'],kernel_regularizer= keras.regularizers.l2(params['l2_ker']), 
                              recurrent_regularizer= keras.regularizers.l2(params['l2_rec']),return_sequences=True)(embedding)
    conv = keras.layers.Conv1D(params['filter'], params['region_size'], activation='relu',kernel_regularizer= keras.regularizers.l2(params['l2_cnn']))(LSTM)
    pool = keras.layers.GlobalMaxPooling1D()(conv)
    output = keras.layers.Dense(2, activation='softmax',kernel_regularizer= keras.regularizers.l2(params['l2_dense']))(pool)
    model  = keras.models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    output = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_data=(x_test, y_test),
                    verbose=0,
                    callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    epoch = int(output.history['val_loss'].index(min(output.history['val_loss']))+1)
    loss, acc = model.evaluate(x_test, y_test)
    talos_optimal_LSTMCNN.loc[len(talos_optimal_LSTMCNN)] = [epoch,loss,acc]
    return output, model

In [26]:
p4 = {'embd_size':[64,100,128],
      'units':[100,150,200],
      'filter':[200,250,300],
      'region_size':[3,4,5],
      'l2_ker':[0.001,0.01],
      'l2_rec':[0.001,0.01],
      'l2_cnn':[0.001,0.01],
      'l2_dense':[0.001,0.01]}

In [27]:
scan_object4 = talos.Scan(x=sd1_data_train,
                         y=sd1_labels_train,
                         params=p4,
                         model=LSTMCNN_model,
                         experiment_name='LSTMCNN_model',
                         x_val= sd1_data_test,
                         y_val= sd1_labels_test,
                         fraction_limit=0.1,
                         random_method= 'quantum')



  0%|          | 0/129 [00:00<?, ?it/s]

12/12 [==============================] - 0s 4ms/step - loss: 0.5214 - accuracy: 0.8568




  1%|          | 1/129 [01:09<2:28:40, 69.69s/it]

12/12 [==============================] - 0s 4ms/step - loss: 0.6181 - accuracy: 0.8621




  2%|▏         | 2/129 [02:36<2:38:35, 74.93s/it]

...



 98%|█████████▊| 127/129 [3:08:40<02:55, 87.61s/it]

12/12 [==============================] - 0s 4ms/step - loss: 0.5558 - accuracy: 0.8647




 99%|█████████▉| 128/129 [3:10:13<01:29, 89.11s/it]

12/12 [==============================] - 0s 4ms/step - loss: 0.4555 - accuracy: 0.8435




100%|██████████| 129/129 [3:11:39<00:00, 89.14s/it]


In [28]:
hasil_LSTMCNN = pd.concat([scan_object4.data,talos_optimal_LSTMCNN],axis=1)
hasil_LSTMCNN.sort_values('val_loss_optimal')

,start,end,duration,round_epochs,loss,accuracy,val_loss,val_accuracy,embd_size,filter,l2_cnn,l2_dense,l2_ker,l2_rec,region_size,units,epochs_optimal,val_loss_optimal,val_acc_optimal
9,11/26/20-142830,11/26/20-143005,95.003282,150,0.074862,0.994695,0.661949,0.862069,128,200,0.010,0.001,0.010,0.001,3,200,125.0,0.407674,0.867374
114,11/26/20-170622,11/26/20-170749,86.909899,150,0.067232,0.998674,0.761970,0.856764,100,200,0.010,0.001,0.010,0.001,3,150,145.0,0.420034,0.870027
15,11/26/20-143713,11/26/20-143840,86.567612,150,0.064540,0.996684,0.707626,0.848806,100,300,0.001,0.010,0.010,0.001,5,100,117.0,0.425144,0.870027
30,11/26/20-150034,11/26/20-150204,90.366581,150,0.065118,0.997347,0.599975,0.870027,100,250,0.010,0.001,0.010,0.001,3,150,89.0,0.434456,0.867374
32,11/26/20-150327,11/26/20-150509,101.474661,150,0.080727,0.994695,0.435829,0.859416,128,200,0.010,0.010,0.001,0.001,5,150,150.0,0.435829,0.859416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,11/26/20-151411,11/26/20-151546,94.442303,150,0.042195,0.997347,0.708254,0.843501,100,300,0.001,0.001,0.001,0.001,5,200,12.0,0.605240,0.859416
76,11/26/20-161120,11/26/20-161251,90.465961,150,0.062309,0.998011,0.695571,0.848806,100,300,0.001,0.010,0.001,0.001,4,200,40.0,0.611912,0.867374
20,11/26/20-144445,11/26/20-144621,95.745106,150,0.044579,0.996684,0.751276,0.840849,128,250,0.001,0.001,0.001,0.001,5,100,9.0,0.613047,0.859416
1,11/26/20-141749,11/26/20-141916,86.930616,150,0.043774,0.996684,0.729632,0.848806,128,250,0.001,0.001,0.001,0.001,5,150,12.0,0.618058,0.862069


In [29]:
hasil_LSTMCNN.sort_values('val_loss_optimal').head(10)

,start,end,duration,round_epochs,loss,accuracy,val_loss,val_accuracy,embd_size,filter,l2_cnn,l2_dense,l2_ker,l2_rec,region_size,units,epochs_optimal,val_loss_optimal,val_acc_optimal
9,11/26/20-142830,11/26/20-143005,95.003282,150,0.074862,0.994695,0.661949,0.862069,128,200,0.010,0.001,0.010,0.001,3,200,125.0,0.407674,0.867374
114,11/26/20-170622,11/26/20-170749,86.909899,150,0.067232,0.998674,0.761970,0.856764,100,200,0.010,0.001,0.010,0.001,3,150,145.0,0.420034,0.870027
15,11/26/20-143713,11/26/20-143840,86.567612,150,0.064540,0.996684,0.707626,0.848806,100,300,0.001,0.010,0.010,0.001,5,100,117.0,0.425144,0.870027
30,11/26/20-150034,11/26/20-150204,90.366581,150,0.065118,0.997347,0.599975,0.870027,100,250,0.010,0.001,0.010,0.001,3,150,89.0,0.434456,0.867374
32,11/26/20-150327,11/26/20-150509,101.474661,150,0.080727,0.994695,0.435829,0.859416,128,200,0.010,0.010,0.001,0.001,5,150,150.0,0.435829,0.859416
71,11/26/20-160343,11/26/20-160524,101.207965,150,0.058553,0.998011,0.627119,0.856764,128,300,0.001,0.001,0.010,0.001,4,150,117.0,0.436340,0.854111
102,11/26/20-164932,11/26/20-165103,90.251105,150,0.057083,0.996684,0.815934,0.843501,100,250,0.010,0.001,0.001,0.001,3,200,55.0,0.437216,0.870027
40,11/26/20-151546,11/26/20-151703,77.247331,150,0.055273,0.996021,0.749732,0.851459,64,300,0.001,0.001,0.010,0.001,3,100,123.0,0.437681,0.854111
88,11/26/20-162835,11/26/20-163012,96.814266,150,0.077918,0.996684,0.622561,0.859416,128,250,0.010,0.001,0.001,0.001,3,150,58.0,0.437837,0.864721
91,11/26/20-163321,11/26/20-163444,82.896852,150,0.124301,0.994695,0.706182,0.856764,64,300,0.010,0.010,0.010,0.001,3,200,147.0,0.438177,0.862069
